In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import data
df = pd.read_csv('data.csv')

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# find the number of missing values in each column
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# remove the rows with missing values in the column 'Daily tests'
df = df.dropna(subset=['Daily tests']).reset_index(drop = True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Replace NaN values before first deaths and cases for each country with 0

In [ ]:
entityGroup = df.groupby('Entity')

for entity, group in entityGroup:
    for index in range(group.index[0], group.index[-1] + 1):
        if (df.isnull().iloc[index]['Cases']):
            df.at[index, 'Cases'] = 0
        else:
            break

In [ ]:
for entity, group in entityGroup:
    for index in range(group.index[0], group.index[-1] + 1):
        if (df.isnull().iloc[index]['Deaths']):
            df.at[index, 'Deaths'] = 0
        else:
            break

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Convert total number of deaths and cases to daily number

In [ ]:
dfCopy = df.copy()
for entity, group in entityGroup:
    for index in range(group.index[0]+1, group.index[-1] + 1):
        df.at[index, 'Cases'] = dfCopy.at[index, 'Cases'] - dfCopy.at[index - 1, 'Cases'] 
        df.at[index, 'Deaths'] = dfCopy.at[index, 'Deaths'] - dfCopy.at[index - 1, 'Deaths']

Create Seasons Column \
1 -> December - February \
2 -> March - May \
3 -> June - August \
4 -> September - November

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'])
df['Season'] = df.Date.dt.month %12 // 3 + 1

In [ ]:
df.head(20)

Graph of Cases by Date

In [ ]:
dfByDate = df.copy()
dfByDate.set_index('Date', inplace=True)

dfByDate = dfByDate.groupby(by=[dfByDate.index]).sum(numeric_only = True)

plt.plot(dfByDate.index, dfByDate['Cases'])
plt.show()

Graph of Deaths by Date

In [ ]:
plt.plot(dfByDate.index, dfByDate['Deaths'])
plt.show()

Graph of Deaths by Season

In [ ]:
plt.title('Deaths by Season')
plt.xlabel('Season')
plt.ylabel('Number of Deaths')
plt.bar(df['Season'], df['Deaths'])

Correlation Heatmap

In [ ]:
sns.heatmap(df.corr(numeric_only=True),annot=True)
plt.show()